In [ ]:
from robmob import robot as rbt
from robmob.sensors import GyroSensor, OdometerTicksSensor
import cv2
import numpy as np

import matplotlib.pyplot as plt

import time
from datetime import datetime

In [ ]:
robot_ip = '192.168.0.100'
robot = rbt.Robot(robot_ip)
robot.connect()

In [ ]:
DELTA_T = 0.1
NUMBER_OF_STEPS = 50

gyro = GyroSensor()
robot.add_sensor(gyro)
print(gyro.peek_data())

odometer = OdometerTicksSensor()
robot.add_sensor(odometer)
print(odometer.peek_data())

In [ ]:
timestamp = []
gyro_data = []
odometer_data = []

for step in range(NUMBER_OF_STEPS):
    timestamp.append(step*DELTA_T)
    gyro_data.append(gyro.peek_data())
    odometer_data.append(odometer.peek_data())
    time.sleep(DELTA_T)
    
np.savetxt('data/log-{}.txt'.format(str(datetime.now())), np.vstack(()))